In [1]:
import lizard
import pandas as pd

In [2]:
comments_read_df = pd.read_pickle("../1.crawling/scraped_files/chromium/comments.pkl")
print(comments_read_df.info())
diff_read_df = pd.read_pickle("../1.crawling/scraped_files/chromium/diff.pkl")
print(diff_read_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  30802 non-null  int64  
 1   change_message_id      30802 non-null  object 
 2   unresolved             30802 non-null  bool   
 3   patch_set              30802 non-null  int64  
 4   id                     30802 non-null  object 
 5   updated                30802 non-null  object 
 6   message                30802 non-null  object 
 7   commit_id              30802 non-null  object 
 8   author._account_id     30802 non-null  int64  
 9   author.name            30802 non-null  object 
 10  author.email           30794 non-null  object 
 11  author.avatars         30802 non-null  object 
 12  author.status          3705 non-null   object 
 13  in_reply_to            16016 non-null  object 
 14  change_id              30802 non-null  object 
 15  pa

In [58]:
# いらないカラムをdropする
comments_df = comments_read_df.loc[:,["id","line","message","range.start_line","range.end_line"]] 
diff_df = diff_read_df.loc[:,["comments_id","content","changes_id","revisions_id","path"]]

In [59]:
diff_df = diff_df[diff_df["comments_id"].duplicated()]
diff_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3274 entries, 3274 to 6551
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comments_id   3274 non-null   object
 1   content       3274 non-null   object
 2   changes_id    3274 non-null   object
 3   revisions_id  3274 non-null   object
 4   path          3274 non-null   object
dtypes: object(5)
memory usage: 153.5+ KB


In [60]:
# 重複削除
comments_df = comments_df[comments_df.duplicated()]
comments_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3342 entries, 3342 to 6687
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3342 non-null   object 
 1   line              3342 non-null   float64
 2   message           3342 non-null   object 
 3   range.start_line  2007 non-null   float64
 4   range.end_line    2007 non-null   float64
dtypes: float64(3), object(2)
memory usage: 156.7+ KB


In [4]:
# revisionにコメントが単一のものだけpick
# up
"""
def filter_one_comment_in_one_revision(comments_df):
  grouped_comments_df = comments_df.groupby("commit_id").size() # commit_idでカウント
  comments_id_sr = grouped_comments_df[grouped_comments_df == 1] # 単一だけのseries
  comments_id_df = pd.DataFrame({"commit_id":comments_id_sr.index})
  result_df = pd.merge(comments_df, comments_id_df, how="inner", on="commit_id")
  return result_df
comments_df = filter_one_comment_in_one_revision(comments_df)
comments_df
"""

,index,change_message_id,unresolved,patch_set,id,updated,message,commit_id,author._account_id,author.name,...,line,range.start_line,range.start_character,range.end_line,range.end_character,side,author.username,author.display_name,author.inactive,tag
0,0,66648aee633913d94580304320c37ad587f1e84f,True,1,9f04e8ec_8547c47e,2021-10-14 12:25:03.000000000,"I think we can drop this check now, as we have...",4bdb71e44f3404baad05a2b8dc596282c1ffc6ad,1150969,Alexander Timin,...,227.0,227.0,0.0,227.0,44.0,NaN,NaN,NaN,NaN,NaN
1,0,fbc8ef0b31cbbc3450ab910b54c9eeb020700eff,False,4,6a7fac5c_f2e42771,2021-10-14 19:43:21.000000000,This kills me how it doesn't match chrome/brow...,50df6cce186ad084a2bc046b6eeda427e2b216d0,1153420,Sky Malice,...,5.0,5.0,36.0,5.0,44.0,NaN,NaN,NaN,NaN,NaN
2,0,bd7940f2e74140ff31f7a0cfe3e68f1eb0043395,False,8,ea466c51_7931b38a,2021-10-14 16:49:02.000000000,This is something new to me. Interesting to kn...,0b50b01c363f3936e4dad038895ee3a7eb2e78da,1373353,Wenyu Fu,...,309.0,309.0,45.0,309.0,54.0,NaN,NaN,NaN,NaN,NaN
3,11,19a65b9069b1bff6b0b5782dfa079b037e215229,True,11,18ef0abb_c1183fd4,2021-10-14 22:56:48.000000000,Is it possible for a renderer process to someh...,dfd36b4ce12f9c5ac925604cf6d157ca0c4bff7b,1001500,Nasko Oskov,...,556.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,05d1ac5a830e6a47bde28f4d1281309fb2d55b2d,True,20,b88925a5_7cb75b24,2021-10-14 19:21:00.000000000,Where does this 1000ms come from? As far as I ...,58a943017db4c00da3cce11e5f053e8ba14c0170,1002411,Michael Thiessen,...,59.0,59.0,7.0,59.0,38.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,0,a379df1e13b7f4a67b49a5e220a5fecf37d587ad,True,1,d7d58a14_e69f7ba9,2021-03-19 19:34:07.000000000,Most of the cfg prints especially are just me ...,401ada330d933cdf37ee6a5b29a108413c1b5fad,1287197,Diana Z,...,561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,12,11cc63a871558954ff90f1c90833cd8c96dc6e32,False,7,b65e7603_636f2095,2018-10-19 09:07:00.000000000,Nice. I hope that at some point we can kill it...,d499494a55a5f8d2f3e26e940924b416108fd625,1153089,Sergiy Belozorov,...,2394.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,0,dba7cc33240fdc9f164e6e28e72595c830b08b7b,True,1,b783f478_97bf49e1,2019-03-13 15:25:18.000000000,nit: rather write:\nremove unmaintained mips b...,5d612627df88110a26b3ba83f551b315f01155e4,1115956,Michael Achenbach,...,7.0,7.0,8.0,7.0,37.0,NaN,NaN,NaN,NaN,NaN
403,0,023d8fd480c0d1571183dd1ea0b1d9e8db5eebf6,False,4,8689e5bc_b4a58ac5,2021-08-12 16:41:32.000000000,IF you know of a good way of adding these duri...,a9892d3bf4af7bdb8e54adb5892449ec4755c0bb,1393955,Goran Flegar,...,829.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# javaファイルに関するdiffのみを抽出する
diff_df = diff_df
diff_df = diff_df[diff_df["path"].str.contains(".java")]
diff_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 3640 to 6469
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comments_id   94 non-null     object
 1   content       94 non-null     object
 2   changes_id    94 non-null     object
 3   revisions_id  94 non-null     object
 4   path          94 non-null     object
dtypes: object(5)
memory usage: 4.4+ KB


In [62]:
# diffのcontentが無いものは削除
diff_df = diff_df[diff_df["content"].apply(lambda x: len(x) != 1)]

In [66]:
# diffとcommentをマージ
comments_df = comments_df.rename(columns={'id': 'comments_id'})
comments_df = pd.merge(comments_df, diff_df)

In [71]:
# javaファイルの作成
def separate_before_after(comment_id,contents):
  before = list()
  after = list()
  for content in contents:
    if "a" in content:
      after.extend(content["a"])
    elif "b" in content:
      before.extend(content["b"])
    elif "ab" in content:
      after.extend(content["ab"])
      before.extend(content["ab"])
  with open("./java_files/"+comment_id+"_before.java","w", encoding="utf-8") as f:
    f.write("\n".join(before))
  with open("./java_files/"+comment_id+"_after.java","w", encoding="utf-8") as f:
    f.write("\n".join(after))
import os
import shutil
shutil.rmtree("./java_files/")
os.mkdir("./java_files/")
[*map(lambda x: separate_before_after(x[0],x[1]), comments_df.loc[:,["comments_id","content"]].values )]
comments_df["filepath_before"] = comments_df["comments_id"].apply(lambda x: x+"_before.java")
comments_df["filepath_after"] = comments_df["comments_id"].apply(lambda x: x+"_after.java")

In [81]:
#  lineからmethod_name_commentedカラムの生成
def read_file_as_list(file_path):
  f = open(file_path, 'r')
  lines = f.readlines()
  f.close()
  return lines

# ファイルをlistで保存
def save_list_as_file(file_path,li):
  with open(file_path,"w", encoding="utf-8") as f:
    f.write("".join(li))

def get_methods_name_commented(filepath,line):
  
  liz = lizard.analyze_file("./java_files/"+filepath)
  for i,liz_func in enumerate(liz.function_list):
    start_line = liz_func.__dict__["start_line"]
    end_line = liz_func.__dict__["end_line"]
    if start_line <= int(line) and line <= end_line:
      return liz_func.__dict__["name"]
  #print("None")
  return "None"

comments_df["method_name_commented"] = comments_df.loc[:,["filepath_before","line"]].apply(lambda x:get_methods_name_commented(x[0],x[1]), axis=1)
comments_df = comments_df[comments_df["method_name_commented"] != "None"]

In [85]:
#  lineからmethod_name_commentedカラムの生成
def is_included_after(filepath,method_name):
  liz = lizard.analyze_file("./java_files/"+filepath)
  for i,liz_func in enumerate(liz.function_list):
    if liz_func.__dict__["name"] == method_name:
      return True
  return False

comments_df["is_included_after"] = comments_df.loc[:,["filepath_after","method_name_commented"]].apply(lambda x:is_included_after(x[0],x[1]), axis=1)
comments_df = comments_df[comments_df["is_included_after"]]

/var/folders/v9/7w9r94gd5qj0kq2g_hr6s4500000gn/T/ipykernel_60212/3276190714.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df["is_included_after"] = comments_df.loc[:,["filepath_after","method_name_commented"]].apply(lambda x:is_included_after(x[0],x[1]), axis=1)


In [93]:
# methodslist_before != method_list_afterかどうかをis_revised_method_commentに反映
# ファイルをlistで保存
def save_list_as_file(file_path,li):
  with open(file_path,"w", encoding="utf-8") as f:
    f.write("".join(li))


def is_included_after(filepath_before,filepath_after,method_name):
  liz_before = lizard.analyze_file("./java_files/"+filepath_before)
  file_before_li = read_file_as_list("./java_files/"+filepath_before)
  liz_after = lizard.analyze_file("./java_files/"+filepath_after)
  file_after_li = read_file_as_list("./java_files/"+filepath_after)
  before_s = ""
  after_s = ""
  for i,liz_func in enumerate(liz_before.function_list):
    if liz_func.__dict__["name"] == method_name:
      start_line = liz_func.__dict__["start_line"]
      end_line = liz_func.__dict__["end_line"]
      methods = file_before_li[start_line-1:end_line]
      before_s = "".join(methods)
      save_list_as_file("./methods/"+filepath_before,methods)
      break
  for i,liz_func in enumerate(liz_after.function_list):
    if liz_func.__dict__["name"] == method_name:
      start_line = liz_func.__dict__["start_line"]
      end_line = liz_func.__dict__["end_line"]
      methods = file_after_li[start_line-1:end_line]
      after_s = "".join(methods)
      save_list_as_file("./methods/"+filepath_after,methods)
      break
  if before_s ==after_s:
    return False
  return True
shutil.rmtree("./methods/")
os.mkdir("./methods/")
comments_df["is_revised_method_comment"] = comments_df.loc[:,["filepath_before","filepath_after","method_name_commented"]].apply(lambda x:is_included_after(x[0],x[1],x[2]), axis=1)
comments_df = comments_df[comments_df["is_revised_method_comment"]]

In [92]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 73
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   comments_id                36 non-null     object 
 1   line                       36 non-null     float64
 2   message                    36 non-null     object 
 3   range.start_line           33 non-null     float64
 4   range.end_line             33 non-null     float64
 5   content                    36 non-null     object 
 6   changes_id                 36 non-null     object 
 7   revisions_id               36 non-null     object 
 8   path                       36 non-null     object 
 9   filepath_before            36 non-null     object 
 10  filepath_after             36 non-null     object 
 11  methods_before_count       36 non-null     int64  
 12  methods_after_count        36 non-null     int64  
 13  method_name_commented      36 non-null     object 
 

In [74]:
# lizardのテスト
i = lizard.analyze_file("test.java")
print(i.__dict__)
for func in i.function_list:
  print( func.__dict__["name"])

{'filename': 'test.java', 'nloc': 1, 'function_list': [<lizard.FunctionInfo object at 0x1b9031eb0>], 'token_count': 34}
VAR_1


In [15]:

diff_df = pd.read_pickle("../1.scraping/scraped_files/chromium/diff.pkl")
diff_df.info()

FileNotFoundError: [Errno 2] No such file or directory: '../1.scraping/scraped_files/chromium/diff.pkl'

In [12]:
comments_df = pd.read_pickle("../1.crawling/scraped_files/chromium/comments.pkl")
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  30802 non-null  int64  
 1   change_message_id      30802 non-null  object 
 2   unresolved             30802 non-null  bool   
 3   patch_set              30802 non-null  int64  
 4   id                     30802 non-null  object 
 5   updated                30802 non-null  object 
 6   message                30802 non-null  object 
 7   commit_id              30802 non-null  object 
 8   author._account_id     30802 non-null  int64  
 9   author.name            30802 non-null  object 
 10  author.email           30794 non-null  object 
 11  author.avatars         30802 non-null  object 
 12  author.status          3705 non-null   object 
 13  in_reply_to            16016 non-null  object 
 14  change_id              30802 non-null  object 
 15  pa

In [3]:
diff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28750 entries, 0 to 28749
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   change_type          28750 non-null  object 
 1   content              28750 non-null  object 
 2   meta_a.name          28229 non-null  object 
 3   meta_a.content_type  28229 non-null  object 
 4   meta_a.lines         28229 non-null  float64
 5   meta_a.web_links     26691 non-null  object 
 6   meta_b.name          28741 non-null  object 
 7   meta_b.content_type  28741 non-null  object 
 8   meta_b.lines         28741 non-null  float64
 9   meta_b.web_links     27203 non-null  object 
 10  changes_id           28750 non-null  object 
 11  revisions_id         28750 non-null  object 
 12  path                 28750 non-null  object 
 13  patch_set_a          28750 non-null  int64  
 14  patch_set_b          28750 non-null  int64  
 15  comments_id          28750 non-null 

In [ ]:
len(diff_df.content[1])

In [ ]:
diff_df["has_diff"] = diff_df["content"].apply(lambda x: True if len(x) != 1 else False)

In [ ]:
diff_df["has_diff"].value_counts()

In [ ]:
diff_df_has_diff = diff_df[diff_df["has_diff"]]

In [ ]:
diff_df_has_diff["is_java"] = diff_df_has_diff["meta_a.name"].str.contains(".java")

In [ ]:
diff_df_has_diff["is_java"].value_counts()

In [ ]:
def separate_before_after(contents,comment_id):
  before = ""
  after = ""
  for content in contents:
    if "a" in content:
      after += content["a"][0] + "\n"
    elif "b" in content:
      before += content["b"][0]+ "\n"
    elif "ab" in content:
      after += content["ab"][0]+ "\n"
      before += content["ab"][0]+ "\n"
  with open(comment_id+"_before","w", encoding="utf-8", newline="\n") as f:
    f.write(before)
  with open(comment_id+"_after","w", encoding="utf-8", newline="\n") as f:
    f.write(after)
separate_before_after(diff_df_has_diff.iloc[0]["content"],diff_df_has_diff.iloc[0]["comments_id"])

In [ ]:
diff_df_has_diff.iloc[0]["content"]

In [ ]:
df = pd.read_pickle("../1.scraping/scraped_files/opendev/diff.pkl")

In [ ]:
df["meta_a.name"].unique()

In [ ]:
df["is_java"] = df["meta_a.name"].str.contains(".java")

In [ ]:
df["is_java"].value_counts()

In [ ]:
diff_df = pd.read_pickle("../1.scraping/scraped_files/chromium/diff.pkl")


In [ ]:
diff_df["has_diff"] = diff_df["content"].apply(lambda x: len(x) != 1)
diff_df["is_java"]  = diff_df["meta_a.name"].str.contains(".java")
df_temp = diff_df[diff_df["is_java"] & diff_df["has_diff"] ].loc[:,["comments_id","content"]].values

In [ ]:
def separate_before_after(comment_id,contents):
  before = list()
  after = list()
  for content in contents:
    if "a" in content:
      after.extend(content["a"])
    elif "b" in content:
      before.extend(content["b"])
    elif "ab" in content:
      after.extend(content["ab"])
      before.extend(content["ab"])
  with open("./files/"+comment_id+"_before.java","w", encoding="utf-8") as f:
    f.write("\n".join(before))
  with open("./files/"+comment_id+"_after.java","w", encoding="utf-8") as f:
    f.write("\n".join(after))


In [ ]:
[*map(lambda x: separate_before_after(x[0],x[1]), df_temp )]

In [ ]:
df_temp = diff_df[diff_df["is_java"] & diff_df["has_diff"] ]

In [ ]:
df_temp.info()

In [ ]:
comments_df = pd.read_pickle("../1.scraping/scraped_files/chromium/comments.pkl")
comments_df.info()

In [ ]:
# merge
comments_df = comments_df.rename(columns={'id': 'comments_id'})
merge_df = pd.merge(comments_df, df_temp)

In [ ]:
merge_df= merge_df.loc[:,["comments_id","line","message","range.start_line","range.end_line"]]

In [ ]:
merge_df["filepath_before"] = merge_df["comments_id"].apply(lambda x: x+"_before.java")
merge_df["filepath_after"] = merge_df["comments_id"].apply(lambda x: x+"_after.java")
merge_df["is_gerrit"] = True
merge_df["repository_name"] = "chromium"




In [ ]:
df = pd.read_pickle("./review_df.pkl")

In [ ]:
df